<a href="https://colab.research.google.com/github/Haniyah14/Youtube-Data-Analysis/blob/main/youtube_ml_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython import get_ipython
from IPython.display import display
# %%
from google.colab import files

uploaded = files.upload() #returns dictionary with filenames as keys.
#values are the filedata

Saving most_subscribed_youtube_channels.csv to most_subscribed_youtube_channels (2).csv


In [ ]:
import pandas as pd
import io

df = pd.read_csv(io.BytesIO(uploaded['most_subscribed_youtube_channels.csv']))

# Clean numerical columns: remove commas and convert to numeric
cols_to_clean = ['subscribers', 'video views', 'video count']
for col in cols_to_clean:
    # Check if the column is of object (string) type before applying .str
    if df[col].dtype == 'object':
        df[col] = df[col].str.replace(',', '', regex=True)
    df[col] = pd.to_numeric(df[col], errors='coerce')  # Convert to numeric, invalid parsing will become NaN

# Drop rows with missing values in key columns
df = df.dropna(subset=cols_to_clean + ['started'])

# Create new features
df['channel_age'] = 2025 - df['started']  # Assuming current year 2025
df['views_per_video'] = df['video views'] / df['video count']
df['subs_per_view'] = df['subscribers'] / df['video views']

# Handle infinite or NaN values in new features
df.replace([float('inf'), -float('inf')], pd.NA, inplace=True)
df.dropna(subset=['views_per_video', 'subs_per_view'], inplace=True)


display(df)

KeyError: 'most_subscribed_youtube_channels.csv'

In [ ]:
import plotly.express as px
fig = px.histogram(df, x='subscribers', title='Subscriber Count')
fig.show()

In [ ]:
import plotly.express as px

fig = px.pie(df, values='subscribers', names='category', title='YouTube Categories')
fig.show()

In [ ]:
import plotly.express as px

fig = px.box(df, y='started', title='Years Started')
fig.show()

Code to clean and prepare data

In [ ]:
import pandas as pd

# Load data
df = pd.read_csv('/mnt/data/most_subscribed_youtube_channels.csv')

# Clean numerical columns: remove commas and convert to numeric
cols_to_clean = ['subscribers', 'video views', 'video count']
for col in cols_to_clean:
    df[col] = df[col].str.replace(',', '', regex=True)
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Drop rows with missing values in key columns
df = df.dropna(subset=cols_to_clean + ['started'])

# Create new features
df['channel_age'] = 2025 - df['started']  # Assuming current year 2025
df['views_per_video'] = df['video views'] / df['video count']
df['subs_per_view'] = df['subscribers'] / df['video views']

# Handle infinite or NaN values in new features
df.replace([float('inf'), -float('inf')], pd.NA, inplace=True)
df.dropna(subset=['views_per_video', 'subs_per_view'], inplace=True)

df.head()


Clustering with K-Means
Code for clustering and visualization

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Select features for clustering
features = ['subscribers', 'video views', 'video count', 'channel_age', 'views_per_video', 'subs_per_view']
X = df[features]

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Find the optimal number of clusters using the elbow method
inertia = []
for k in range(1, 10):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_scaled)
    inertia.append(kmeans.inertia_)

plt.plot(range(1, 10), inertia, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.title('Elbow Method For Optimal k')
plt.show()


Apply KMeans with chosen k (e.g., k=3)

In [ ]:
k = 3  # Change based on elbow method result
kmeans = KMeans(n_clusters=k, random_state=42)
df['cluster'] = kmeans.fit_predict(X_scaled)

# Inspect cluster counts
print(df['cluster'].value_counts())


Visualize clusters using PCA (2D)
python
Copy


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

plt.figure(figsize=(8,6))
scatter = plt.scatter(X_pca[:,0], X_pca[:,1], c=df['cluster'], cmap='viridis')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.title('Clusters of YouTube Channels')
plt.legend(*scatter.legend_elements(), title='Clusters')
plt.show()


Regression to Predict Subscribers -
Code for simple Linear Regression

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Features & target
X = df[['video views', 'video count', 'channel_age', 'views_per_video', 'subs_per_view']]
y = df['subscribers']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
lr = LinearRegression()
lr.fit(X_train, y_train)

# Predictions
y_pred = lr.predict(X_test)

# Evaluation
print('RMSE:', mean_squared_error(y_test, y_pred, squared=False))
print('R^2:', r2_score(y_test, y_pred))
